# Set up

In [8]:
from huggingface_hub import login
login("YOUR KEY HERE")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [9]:
import json
import os
import chromadb
from langchain_chroma import Chroma

from langchain.text_splitter import RecursiveCharacterTextSplitter
from llm.llm import BedRockLLMs,CoreLLMs
from llm.llm_utils import *
import numpy as np
from pyvi import ViTokenizer
import boto3
from langchain.embeddings.openai import OpenAIEmbeddings
from prompts import *

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [12]:
llm = CoreLLMs(quantization='int4')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

## Load Database


In [15]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [ ]:
model = 'bkai-foundation-models/vietnamese-bi-encoder'
embd = SentenceTransformerEmbeddings(model_name=model)

## Original data Vector Database

In [1]:
data_dir="data_raw10k"
persist_dir="/content/drive/MyDrive/speech/RAG/ChromaDB/chromadb"
col_name="wiki10k"
DATA_DIR = data_dir
CHROMA_PATH = persist_dir
COLLECTION_NAME = col_name
vectorstore1 = Chroma(collection_name=COLLECTION_NAME, persist_directory=CHROMA_PATH, embedding_function=embd)

## Synthetic data Vector Database

In [17]:
data_dir="gendata"
persist_dir="/content/drive/MyDrive/speech/RAG/ChromaDB/genchromadb/"
col_name="genwiki10k"
DATA_DIR = data_dir
CHROMA_PATH = persist_dir
COLLECTION_NAME = col_name
vectorstore2 = Chroma(collection_name=COLLECTION_NAME, persist_directory=CHROMA_PATH, embedding_function=embd)

In [18]:
vectorstore2.similarity_search("Ngày nào Graham Martin bị sơ tán khỏi Việt Nam năm 1975?")

[Document(page_content='Dưới đây là một tóm tắt về Graham A. Martin - Đại sứ Hoa Kỳ cuối cùng tại Việt Nam Cộng hòa:\n\n- Graham A. Martin (1912-1990) là một nhà ngoại giao và chính trị gia Mỹ. Ông kế nhiệm Ellsworth Bunker làm Đại sứ Mỹ tại Việt Nam Cộng hòa từ năm 1973 và là Đại sứ cuối cùng tại đây.\n\n- Martin sinh ra tại Mars Hill, North Carolina và tốt nghiệp Đại học Wake Forest năm 1932. Ông từng làm phóng viên, rồi làm việc cho chính quyền Tổng thống Roosevelt thời Đại Suy Thoái.\n\n- Trong Chiến tranh Thế giới thứ 2, ông là sĩ quan tình báo, giúp báo cáo tình hình chiến sự ở Thái Bình Dương cho Bộ trưởng Quốc phòng và Bộ trưởng Ngoại giao Mỹ.  \n\n- Sau chiến tranh, ông bắt đầu sự nghiệp ngoại giao, có thời gian dài công tác tại Paris quan sát phong trào cộng sản. Năm 1963, ông được bổ nhiệm làm Đại sứ tại Thái Lan.\n\n- Tại Việt Nam, Martin đánh giá thấp tình hình và tin tưởng quân đội VNCH có thể giữ vững miền Nam. Ông là Đại sứ cuối cùng được đưa đi bằng trực thăng khi Sài 

## RAG

### Load questions

In [22]:
questions = json.loads(open('/content/drive/MyDrive/speech/RAG/hard_questions_4000.json').read())

In [23]:
def RAG(questions):
    data = vectorstore1.similarity_search(questions,5)
    context = '\n\n'.join([doc.page_content for doc in data])
    return context

## Chat

In [24]:
ct = RAG("Amee là ai?")

In [27]:
t = RAG_QA_TL(llm, "Amee là ai?", ct)

In [32]:
print(t)

Here is the answer:

[{"đoạn văn": "Amee là một nữ ca sĩ kiêm diễn viên người Việt Nam, sinh ngày 23 tháng 3 năm 2000. Cô hiện đang là nghệ sĩ trực thuộc ST.319 Entertainment. Amee đã ra mắt với album đầu tay 'dreAmee' vào năm 2020 và đã nhận được nhiều giải thưởng, bao gồm Giải thưởng Âm nhạc Châu Á Mnet 2020 và Giải thưởng Làn Sóng Xanh 2021."}]
